In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re as re
import nltk as nl
from nltk import word_tokenize, pos_tag, bigrams
from nltk.parse.api import ParserI
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

sns.set
pd.options.display.max_colwidth = 200

In [2]:
data_raw = pd.read_csv('C:\\Users\\DELL\\Desktop\\Akshay\\Datasets\\factoid_questions_classification\\factoid_questions_dataset.txt', sep='delimiter', header=None, names=['Factoids'], dtype={'Factoids':str})

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
data_raw

,Factoids
0,DESC:manner How did serfdom develop in and then leave Russia ?
1,ENTY:cremat What films featured the character Popeye Doyle ?
2,DESC:manner How can I find a list of celebrities ' real names ?
3,ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?
4,ABBR:exp What is the full form of .com ?
...,...
995,DESC:manner How do you look up criminal records on the Internet ?
996,"LOC:other Where is the actress , Marion Davies , buried ?"
997,DESC:desc What is an example of an actual case of using the Fourteenth Amendment ?
998,HUM:ind Who was the Charlie perfume woman ?


In [4]:
data_raw['Factoids'] = data_raw.Factoids.str.replace(':', ' ')
data_raw['Class'] = data_raw.Factoids.str.split('\s+').str[0]
data_raw['Sub-Class'] = data_raw.Factoids.str.split('\s+').str[1]
data_raw['Questions'] = data_raw.Factoids.str.split().str[2:].str.join(' ')
data = data_raw.drop('Factoids', axis=1)
data

,Class,Sub-Class,Questions
0,DESC,manner,How did serfdom develop in and then leave Russia ?
1,ENTY,cremat,What films featured the character Popeye Doyle ?
2,DESC,manner,How can I find a list of celebrities ' real names ?
3,ENTY,animal,What fowl grabs the spotlight after the Chinese Year of the Monkey ?
4,ABBR,exp,What is the full form of .com ?
...,...,...,...
995,DESC,manner,How do you look up criminal records on the Internet ?
996,LOC,other,"Where is the actress , Marion Davies , buried ?"
997,DESC,desc,What is an example of an actual case of using the Fourteenth Amendment ?
998,HUM,ind,Who was the Charlie perfume woman ?


In [5]:
punct_signs = list("?:!.,;")
data_raw['Questions_Parsed_1'] = data_raw["Questions"]

for punct_sign in punct_signs:
    data_raw['Questions_Parsed_1'] = data_raw.Questions_Parsed_1.str.replace(punct_sign, '')

data_raw['Questions_Parsed_1'] = data_raw.Questions_Parsed_1.str.replace('-', ' ')
data_raw['Questoins_Parsed_1'] = data_raw.Questions_Parsed_1.str.replace("'s", "")
#data_raw['Questions_Parsed_1'] = data_raw.Questions_Parsed_1.str.replace('"', '')
data_raw

,Factoids,Class,Sub-Class,Questions,Questions_Parsed_1,Questoins_Parsed_1
0,DESC manner How did serfdom develop in and then leave Russia ?,DESC,manner,How did serfdom develop in and then leave Russia ?,How did serfdom develop in and then leave Russia,How did serfdom develop in and then leave Russia
1,ENTY cremat What films featured the character Popeye Doyle ?,ENTY,cremat,What films featured the character Popeye Doyle ?,What films featured the character Popeye Doyle,What films featured the character Popeye Doyle
2,DESC manner How can I find a list of celebrities ' real names ?,DESC,manner,How can I find a list of celebrities ' real names ?,How can I find a list of celebrities ' real names,How can I find a list of celebrities ' real names
3,ENTY animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?,ENTY,animal,What fowl grabs the spotlight after the Chinese Year of the Monkey ?,What fowl grabs the spotlight after the Chinese Year of the Monkey,What fowl grabs the spotlight after the Chinese Year of the Monkey
4,ABBR exp What is the full form of .com ?,ABBR,exp,What is the full form of .com ?,What is the full form of com,What is the full form of com
...,...,...,...,...,...,...
995,DESC manner How do you look up criminal records on the Internet ?,DESC,manner,How do you look up criminal records on the Internet ?,How do you look up criminal records on the Internet,How do you look up criminal records on the Internet
996,"LOC other Where is the actress , Marion Davies , buried ?",LOC,other,"Where is the actress , Marion Davies , buried ?",Where is the actress Marion Davies buried,Where is the actress Marion Davies buried
997,DESC desc What is an example of an actual case of using the Fourteenth Amendment ?,DESC,desc,What is an example of an actual case of using the Fourteenth Amendment ?,What is an example of an actual case of using the Fourteenth Amendment,What is an example of an actual case of using the Fourteenth Amendment
998,HUM ind Who was the Charlie perfume woman ?,HUM,ind,Who was the Charlie perfume woman ?,Who was the Charlie perfume woman,Who was the Charlie perfume woman


In [6]:
data_raw['Questions_Parsed_2'] = data_raw.Questions_Parsed_1.str.lower()
data_raw

,Factoids,Class,Sub-Class,Questions,Questions_Parsed_1,Questoins_Parsed_1,Questions_Parsed_2
0,DESC manner How did serfdom develop in and then leave Russia ?,DESC,manner,How did serfdom develop in and then leave Russia ?,How did serfdom develop in and then leave Russia,How did serfdom develop in and then leave Russia,how did serfdom develop in and then leave russia
1,ENTY cremat What films featured the character Popeye Doyle ?,ENTY,cremat,What films featured the character Popeye Doyle ?,What films featured the character Popeye Doyle,What films featured the character Popeye Doyle,what films featured the character popeye doyle
2,DESC manner How can I find a list of celebrities ' real names ?,DESC,manner,How can I find a list of celebrities ' real names ?,How can I find a list of celebrities ' real names,How can I find a list of celebrities ' real names,how can i find a list of celebrities ' real names
3,ENTY animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?,ENTY,animal,What fowl grabs the spotlight after the Chinese Year of the Monkey ?,What fowl grabs the spotlight after the Chinese Year of the Monkey,What fowl grabs the spotlight after the Chinese Year of the Monkey,what fowl grabs the spotlight after the chinese year of the monkey
4,ABBR exp What is the full form of .com ?,ABBR,exp,What is the full form of .com ?,What is the full form of com,What is the full form of com,what is the full form of com
...,...,...,...,...,...,...,...
995,DESC manner How do you look up criminal records on the Internet ?,DESC,manner,How do you look up criminal records on the Internet ?,How do you look up criminal records on the Internet,How do you look up criminal records on the Internet,how do you look up criminal records on the internet
996,"LOC other Where is the actress , Marion Davies , buried ?",LOC,other,"Where is the actress , Marion Davies , buried ?",Where is the actress Marion Davies buried,Where is the actress Marion Davies buried,where is the actress marion davies buried
997,DESC desc What is an example of an actual case of using the Fourteenth Amendment ?,DESC,desc,What is an example of an actual case of using the Fourteenth Amendment ?,What is an example of an actual case of using the Fourteenth Amendment,What is an example of an actual case of using the Fourteenth Amendment,what is an example of an actual case of using the fourteenth amendment
998,HUM ind Who was the Charlie perfume woman ?,HUM,ind,Who was the Charlie perfume woman ?,Who was the Charlie perfume woman,Who was the Charlie perfume woman,who was the charlie perfume woman


In [7]:
toks = [word_tokenize(question) for question in data_raw.Questions_Parsed_2]

In [8]:
toks

[['how', 'did', 'serfdom', 'develop', 'in', 'and', 'then', 'leave', 'russia'],
 ['what', 'films', 'featured', 'the', 'character', 'popeye', 'doyle'],
 ['how',
  'can',
  'i',
  'find',
  'a',
  'list',
  'of',
  'celebrities',
  "'",
  'real',
  'names'],
 ['what',
  'fowl',
  'grabs',
  'the',
  'spotlight',
  'after',
  'the',
  'chinese',
  'year',
  'of',
  'the',
  'monkey'],
 ['what', 'is', 'the', 'full', 'form', 'of', 'com'],
 ['what',
  'contemptible',
  'scoundrel',
  'stole',
  'the',
  'cork',
  'from',
  'my',
  'lunch'],
 ['what', 'team', 'did', 'baseball', "'s", 'st', 'louis', 'browns', 'become'],
 ['what', 'is', 'the', 'oldest', 'profession'],
 ['what', 'are', 'liver', 'enzymes'],
 ['name',
  'the',
  'scar',
  'faced',
  'bounty',
  'hunter',
  'of',
  'the',
  'old',
  'west'],
 ['when', 'was', 'ozzy', 'osbourne', 'born'],
 ['why', 'do', 'heavier', 'objects', 'travel', 'downhill', 'faster'],
 ['who', 'was', 'the', 'pride', 'of', 'the', 'yankees'],
 ['who', 'killed', 'g

In [9]:
postagged = [pos_tag(tok) for tok in toks]

pos_tagss = []

for i in range(len(postagged)):
    postagged[i]
    tags = []
    for j in range(len(postagged[i])):
        tags.append(postagged[i][j][1])
    pos_tagss.append(tags)

pos_tagss

[['WRB', 'VBD', 'VB', 'VB', 'IN', 'CC', 'RB', 'VB', 'NN'],
 ['WP', 'VBD', 'VBD', 'DT', 'NN', 'NN', 'NN'],
 ['WRB', 'MD', 'VB', 'VB', 'DT', 'NN', 'IN', 'NNS', 'POS', 'JJ', 'NNS'],
 ['WP', 'NN', 'VBD', 'DT', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN'],
 ['WP', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'NN'],
 ['WP', 'JJ', 'NN', 'VBD', 'DT', 'NN', 'IN', 'PRP$', 'NN'],
 ['WP', 'NN', 'VBD', 'NN', 'POS', 'JJ', 'NN', 'NNS', 'VBP'],
 ['WP', 'VBZ', 'DT', 'JJS', 'NN'],
 ['WDT', 'VBP', 'JJ', 'NNS'],
 ['NN', 'DT', 'NN', 'VBD', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN'],
 ['WRB', 'VBD', 'JJ', 'JJ', 'NN'],
 ['WRB', 'VBP', 'JJR', 'NNS', 'VBP', 'NN', 'RBR'],
 ['WP', 'VBD', 'DT', 'NN', 'IN', 'DT', 'NNS'],
 ['WP', 'VBD', 'NN'],
 ['WP', 'VBZ', 'VBN', 'DT', 'JJS', 'NN', 'DT', 'NN', 'NN', 'VBZ', 'RB', 'VBN'],
 ['WP', 'VBG', 'PRP', 'NN', 'VBZ', 'DT', 'RBS', 'NNS'],
 ['WDT', 'VBD', 'DT', 'RB', 'JJ', 'NN', 'TO', 'DT', 'PRP', 'NN', 'NN', 'IN'],
 ['WRB', 'JJ', 'NNS', 'VBD', 'VBN', 'IN', 'NN', 'NNS', 'IN', 'NN'],
 ['WP', 'VBZ

In [10]:
tags_shape = []

for pos_tags in pos_tagss:
    temp = len(pos_tags)
    
    tags_shape.append(temp)
    
max(tags_shape)

30

In [40]:
def sequence_to_2d_pos_tags(pos_tags):
    tags = pos_tags+["<PAD>"]*(30-len(pos_tags)) if len(pos_tags) < 30 else pos_tags
    
    return [
        0 if tag == "<PAD>" else (1 if tag != "<PAD>" else 0) for tag in tags
    ]

In [41]:
tags = pos_tagss[0] + ["<PAD>"]*(30-len(pos_tagss[0]))

test_features = np.array([0 if x1 == "<PAD>" else (1 if x1 != "<PAD>" else 0) for x1 in tags])
test_df = pd.DataFrame(test_features)

print(tags)
print(test_features)

['WRB', 'VBD', 'VB', 'VB', 'IN', 'CC', 'RB', 'VB', 'NN', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [42]:
features_1 = np.array([sequence_to_2d_pos_tags(pos_tags) for pos_tags in pos_tagss])
features_1

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [43]:
features = pd.DataFrame(features_1)

features

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
996,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
998,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
max_len = 37

text = "How far is it from Denver to Aspen?"

tok_text = word_tokenize(text)
print(f"Tokens: {tok_text}")

postag_text = pos_tag(tok_text)
print(f"Pos Tags: {list(postag_text)}")

t = tok_text+["<PAD>"]*(37-len(tok_text)) if len(tok_text) < 37 else tok_text
print(f"Padded Text: {t}")

Tokens: ['How', 'far', 'is', 'it', 'from', 'Denver', 'to', 'Aspen', '?']
Pos Tags: [('How', 'WRB'), ('far', 'RB'), ('is', 'VBZ'), ('it', 'PRP'), ('from', 'IN'), ('Denver', 'NNP'), ('to', 'TO'), ('Aspen', 'VB'), ('?', '.')]
Padded Text: ['How', 'far', 'is', 'it', 'from', 'Denver', 'to', 'Aspen', '?', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [61]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'

cp = nl.RegexpParser(pattern)
cs = cp.parse(postag_text)

cs

The Ghostscript executable isn't found.
See http://web.mit.edu/ghostscript/www/Install.htm
If you're using a Mac, you can try installing
https://docs.brew.sh/Installation then `brew install ghostscript`


LookupError: 

Tree('S', [('How', 'WRB'), ('far', 'RB'), ('is', 'VBZ'), ('it', 'PRP'), ('from', 'IN'), ('Denver', 'NNP'), ('to', 'TO'), ('Aspen', 'VB'), ('?', '.')])

In [62]:
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint

iob_tagged = tree2conlltags(cs)

pprint(iob_tagged)

[('How', 'WRB', 'O'),
 ('far', 'RB', 'O'),
 ('is', 'VBZ', 'O'),
 ('it', 'PRP', 'O'),
 ('from', 'IN', 'O'),
 ('Denver', 'NNP', 'O'),
 ('to', 'TO', 'O'),
 ('Aspen', 'VB', 'O'),
 ('?', '.', 'O')]


In [8]:
features = np.array([list_to_array_pos_tag(pos_tag) for pos_tag in pos_tags])
features

array([['WRB', 'VBD', 'VB', ..., '-', '-', '-'],
       ['WP', 'VBD', 'VBD', ..., '-', '-', '-'],
       ['WRB', 'MD', 'PRP', ..., '-', '-', '-'],
       ...,
       ['WP', 'VBZ', 'DT', ..., '-', '-', '-'],
       ['WP', 'VBD', 'DT', ..., '-', '-', '-'],
       ['WP', 'VBD', '``', ..., '-', '-', '-']], dtype='<U4')

In [9]:
a = pd.DataFrame(features)
a

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,WRB,VBD,VB,VB,IN,CC,RB,VB,NNP,.,...,-,-,-,-,-,-,-,-,-,-
1,WP,VBD,VBD,DT,NN,NNP,NNP,.,-,-,...,-,-,-,-,-,-,-,-,-,-
2,WRB,MD,PRP,VB,DT,NN,IN,NNS,POS,JJ,...,-,-,-,-,-,-,-,-,-,-
3,WP,NN,VBD,DT,NN,IN,DT,JJ,NN,IN,...,-,-,-,-,-,-,-,-,-,-
4,WP,VBZ,DT,JJ,NN,IN,NN,.,-,-,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,WRB,VB,PRP,VB,RP,JJ,NNS,IN,DT,NNP,...,-,-,-,-,-,-,-,-,-,-
996,WRB,VBZ,DT,NN,",",NNP,NNP,",",VBN,.,...,-,-,-,-,-,-,-,-,-,-
997,WP,VBZ,DT,NN,IN,DT,JJ,NN,IN,VBG,...,-,-,-,-,-,-,-,-,-,-
998,WP,VBD,DT,NNP,NN,NN,.,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(1,1))
dtm = vect.fit_transform(data.Questions)

a= pd.DataFrame(dtm.toarray(), columns=vect.get_feature_names())
a

,000,10,103,11,118,12,126,134,14,142,...,you,young,youngest,your,yousuf,zealand,zenger,zodiacal,zones,zorro
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(ngram_range=(1,1))
dtm = vect.fit_transform(features)

dtm

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [19]:
a.columns.to_list()

['000',
 '10',
 '103',
 '11',
 '118',
 '12',
 '126',
 '134',
 '14',
 '142',
 '15',
 '158',
 '167',
 '16th',
 '177',
 '1797',
 '1832',
 '185',
 '1866',
 '1872',
 '187s',
 '1893',
 '1899',
 '19',
 '1920s',
 '1926',
 '1929',
 '193',
 '1932',
 '1939',
 '1941',
 '1942',
 '1948',
 '195',
 '1958',
 '1960',
 '1964',
 '1965',
 '1966',
 '1967',
 '1969',
 '197',
 '1972',
 '1973',
 '1978',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1989',
 '1990s',
 '1991',
 '1993',
 '1994',
 '1998',
 '1999',
 '19th',
 '21',
 '219',
 '22',
 '24',
 '26',
 '280',
 '401',
 '42',
 '468',
 '49',
 '54c',
 '55',
 '5th',
 '6th',
 '84',
 '85',
 '86ed',
 '98',
 'abbreviated',
 'abbreviation',
 'abominable',
 'about',
 'academy',
 'access',
 'accessory',
 'accidents',
 'accompanying',
 'according',
 'account',
 'acetylsalicylic',
 'acid',
 'acreage',
 'act',
 'acted',
 'actor',
 'actress',
 'actual',
 'ad',
 'adding',
 'address',
 'addresses',
 'admiral',
 'admit',
 'admonition',
 'ads',
 'adult',
 'adv

In [10]:
data = data.join(a)
data

,Class,Sub-Class,Questions,0,1,2,3,4,5,6,...,27,28,29,30,31,32,33,34,35,36
0,DESC,manner,How did serfdom develop in and then leave Russia ?,WRB,VBD,VB,VB,IN,CC,RB,...,-,-,-,-,-,-,-,-,-,-
1,ENTY,cremat,What films featured the character Popeye Doyle ?,WP,VBD,VBD,DT,NN,NNP,NNP,...,-,-,-,-,-,-,-,-,-,-
2,DESC,manner,How can I find a list of celebrities ' real names ?,WRB,MD,PRP,VB,DT,NN,IN,...,-,-,-,-,-,-,-,-,-,-
3,ENTY,animal,What fowl grabs the spotlight after the Chinese Year of the Monkey ?,WP,NN,VBD,DT,NN,IN,DT,...,-,-,-,-,-,-,-,-,-,-
4,ABBR,exp,What is the full form of .com ?,WP,VBZ,DT,JJ,NN,IN,NN,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,DESC,manner,How do you look up criminal records on the Internet ?,WRB,VB,PRP,VB,RP,JJ,NNS,...,-,-,-,-,-,-,-,-,-,-
996,LOC,other,"Where is the actress , Marion Davies , buried ?",WRB,VBZ,DT,NN,",",NNP,NNP,...,-,-,-,-,-,-,-,-,-,-
997,DESC,desc,What is an example of an actual case of using the Fourteenth Amendment ?,WP,VBZ,DT,NN,IN,DT,JJ,...,-,-,-,-,-,-,-,-,-,-
998,HUM,ind,Who was the Charlie perfume woman ?,WP,VBD,DT,NNP,NN,NN,.,...,-,-,-,-,-,-,-,-,-,-


In [11]:
features = []
for i in range(len(questions)):
    postagged[i]
    
    question_features = np.array(())
    question_count = []
    token_count = []
    tokens = []
    postags = []
    
    for j in range(len(postagged[i])):
        question_count.append(i)
        token_count.append(j)
        tokens.append(postagged[i][j][0])
        postags.append(postagged[i][j][1])
    
    a = np.array(question_count)
    b = np.array(token_count)
    c = np.array(tokens)
    d = np.array(postags)
    
    question_features = np.column_stack((a,b,c,d))
    features.append(question_features)

features = np.array(features)

NameError: name 'questions' is not defined

In [14]:
data.columns

Index([    'Class', 'Sub-Class', 'Questions',           0,           1,
                 2,           3,           4,           5,           6,
                 7,           8,           9,          10,          11,
                12,          13,          14,          15,          16,
                17,          18,          19,          20,          21,
                22,          23,          24,          25,          26,
                27,          28,          29,          30,          31,
                32,          33,          34,          35,          36],
      dtype='object')

In [15]:
targets = data['Class']
features = data.drop(['Class', 'Sub-Class', 'Questions'], axis=1)

In [16]:
features

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,WRB,VBD,VB,VB,IN,CC,RB,VB,NNP,.,...,-,-,-,-,-,-,-,-,-,-
1,WP,VBD,VBD,DT,NN,NNP,NNP,.,-,-,...,-,-,-,-,-,-,-,-,-,-
2,WRB,MD,PRP,VB,DT,NN,IN,NNS,POS,JJ,...,-,-,-,-,-,-,-,-,-,-
3,WP,NN,VBD,DT,NN,IN,DT,JJ,NN,IN,...,-,-,-,-,-,-,-,-,-,-
4,WP,VBZ,DT,JJ,NN,IN,NN,.,-,-,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,WRB,VB,PRP,VB,RP,JJ,NNS,IN,DT,NNP,...,-,-,-,-,-,-,-,-,-,-
996,WRB,VBZ,DT,NN,",",NNP,NNP,",",VBN,.,...,-,-,-,-,-,-,-,-,-,-
997,WP,VBZ,DT,NN,IN,DT,JJ,NN,IN,VBG,...,-,-,-,-,-,-,-,-,-,-
998,WP,VBD,DT,NNP,NN,NN,.,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(1,1), stop_words='english')
dtm = vect.fit_transform(data.questions)

dtm

AttributeError: 'DataFrame' object has no attribute 'questions'

In [17]:
x_train, x_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=420)

In [18]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)

#Validating Model
acc_logreg = round(accuracy_score(y_pred, y_test) * 100, 2)
pre_logreg = round(precision_score(y_pred, y_test, average='macro') * 100, 2)
rec_logreg = round(recall_score(y_pred, y_test, average='macro') * 100, 2)

print(f"Model Performance: \nAccuracy = {acc_logreg} \nPrecision = {pre_logreg} \nRecall = {rec_logreg}")

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: '-'